In [1]:
import os
import io
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.storage.blob import ContainerClient

In [2]:
# Load all environment variables
load_dotenv()
account_url = os.getenv('AZ_STORAGE_EP')
container_name = os.getenv('AZ_STORAGE_CONTAINER')

In [3]:
# Authenticate
default_credential = DefaultAzureCredential()

For the first time, you might need to authenticate via the Azure CLI

Download it from https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-windows?tabs=azure-cli

Install then run `az login` in your terminal. Once you have logged in with your browser try the `DefaultAzureCredential()` again!

In [4]:
# Connect to container
container_client = ContainerClient(account_url, container_name, default_credential)

In [5]:
# List files in container - should be empty
blob_list = container_client.list_blob_names()
for blob in blob_list:
    if blob.startswith('newdir'):
        print(blob)

newdir/cats.parquet
newdir/ronald.jpeg


In [6]:
# Upload file to container
with open(file='data/cats.csv', mode="rb") as data:
    blob_client = container_client.upload_blob(name='newdir/cats.csv', 
                                               data=data, 
                                               overwrite=True)

In [7]:
# # Check files have uploaded - List files in container again
blob_list = container_client.list_blobs()
for blob in blob_list:
    if blob['name'].startswith('newdir'):
        print(blob['name'])

newdir/cats.csv
newdir/cats.parquet
newdir/ronald.jpeg


In [8]:
# Download file from Azure container to temporary filepath

# Connect to blob
blob_client = container_client.get_blob_client('newdir/cats.csv')

# Write to local file from blob
temp_filepath = os.path.join('temp_data', 'cats.csv')
with open(file=temp_filepath, mode="wb") as sample_blob:
    download_stream = blob_client.download_blob()
    sample_blob.write(download_stream.readall())
cat_data = pd.read_csv(temp_filepath)
cat_data.head()

,Name,Physical_characteristics,Behaviour
0,Ronald,White and ginger,Lazy and greedy but undoubtedly cutest and best
1,Kaspie,Small calico,Sweet and very shy but adventurous
2,Hennimore,Pale orange,Unhinged and always in a state of panic
3,Thug cat,Black and white - very large,Local bully
4,Son of Stripey,Grey tabby,Very vocal


In [9]:
# Load directly from Azure - no local copy

download_stream = blob_client.download_blob()
stream_object = io.BytesIO(download_stream.readall())
cat_data = pd.read_csv(stream_object)
cat_data

,Name,Physical_characteristics,Behaviour
0,Ronald,White and ginger,Lazy and greedy but undoubtedly cutest and best
1,Kaspie,Small calico,Sweet and very shy but adventurous
2,Hennimore,Pale orange,Unhinged and always in a state of panic
3,Thug cat,Black and white - very large,Local bully
4,Son of Stripey,Grey tabby,Very vocal


In [10]:
# !!!!!!!!! Delete from Azure container !!!!!!!!!
blob_client = container_client.get_blob_client('newdir/cats.csv')
blob_client.delete_blob()

In [11]:
blob_list = container_client.list_blobs()
for blob in blob_list:
    if blob['name'].startswith('newdir'):
        print(blob['name'])

newdir/cats.parquet
newdir/ronald.jpeg
